In [20]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pandas as pd
from database import Persistence

base_link = 'https://pm.gc.ca/en/videos?page=0'
root = 'https://pm.gc.ca'

In [2]:
base_html = BeautifulSoup(requests.get(base_link).content, 'lxml')
last_link = base_html.find('li', {'class':'pager__item pager__item--last'}).find('a').get('href')
last_page_num = re.search(r'[0-9]+', last_link).group()

In [3]:
video_links = []
for i in range(0, int(last_page_num) + 1):
    num = str(i)
    data = requests.get(f'https://pm.gc.ca/en/videos/get_vids?page={num}').json()['vids']
    for video in data:
        video_links.append(root + video['url'])

In [4]:
def filter_text(text):
    return text.replace('\xa0', '').replace('\n', '')


def get_text(link):
    soup = BeautifulSoup(requests.get(link).content, 'lxml')
    title = soup.find('div', {'class':'title-header-inner'}).text
    soup_text = soup.find('div', {'class':'content'}).find_all('div')
    if len(soup_text)==2:
        soup_text = filter_text(soup_text[1].text)
    else:
        soup_text = ''
    return {
        'title': filter_text(title),
        'text':soup_text
    }  


In [10]:
data = [get_text(x) for x in tqdm(video_links)]

100%|██████████| 638/638 [09:29<00:00,  1.12it/s]


In [17]:
data = pd.DataFrame(data).rename(columns={'text':'video_text'}).to_dict('records')

In [22]:
Persistence.write_pm_vid_data(data, 'pm_video_text_sam')
print('done!')

done!
